In [5]:
from flask import Flask, render_template, request
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer 
import re
import pickle

In [6]:
stemmer = PorterStemmer()
sia = SentimentIntensityAnalyzer()
vec = pickle.load(open("vectorizer.pkl","rb"))
selector = pickle.load(open("selector.pkl","rb"))
model = pickle.load(open("voting_Model.pkl","rb"))

C:\Users\91738\AppData\Local\Temp\ipykernel_10364\78923589.py:3: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  vec = pickle.load(open("vectorizer.pkl","rb"))


In [7]:
def textProcessing_Vectorizing(text):
    text = text.lower()
    text = re.sub("^[a-z]"," ",text)
    text = [stemmer.stem(word) for word in nltk.word_tokenize(text) if word not in stopwords.words("english")]
    text = " ".join(text)
    vec_text = vec.transform([text]).toarray()
    vec_text = selector.transform(vec_text)
    return vec_text
def polarityScores(text):
    sentimentScores = sia.polarity_scores(text)
    if sentimentScores["compound"]<0:
        return "negative"
    elif sentimentScores["compound"]==0:
        return "neutral"
    elif sentimentScores["compound"]>0:
        return "positive"

In [8]:
app = Flask(__name__)

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    text = request.form['text']
    vec_text = textProcessing_Vectorizing(text)
    model_prediction = model.predict(vec_text)[0]
    vader_prediction = polarityScores(text)
    if vader_prediction=="positive":
        definition="A text that expresses a favorable or optimistic view towards a subject or event.\
        It may convey happiness, satisfaction, excitement, admiration, or other positive emotions."
    elif vader_prediction=="negative":
        definition="A text that expresses an unfavorable or pessimistic view towards a subject or event.\
        It may convey sadness, anger, disappointment, frustration, or other negative emotions."
    elif vader_prediction=="neutral":
        definition="A text that expresses a factual or objective view towards a subject or event without \
        showing any emotional bias. It may provide information or describe something without conveying any \
        particular feeling or judgment."
    return render_template('index.html',head=vader_prediction.upper(),prediction1=model_prediction,prediction2=vader_prediction,definition=definition)

if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [30/Apr/2023 11:15:29] "GET / HTTP/1.1" 200 -
C:\Python310\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
127.0.0.1 - - [30/Apr/2023 11:15:37] "POST /predict HTTP/1.1" 200 -
